In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [2]:

batch_size = 128
num_files = 2
embedding_size = 300
question_size = 30


In [3]:
def get_concatenated_embeddings(temp_df):
#        print("tdf", temp_df.shape)
        zero_embeddings = np.zeros(embedding_size)
        truncated_df = temp_df[:question_size]
#        print(truncated_df.shape)
#        print(len(truncated_df),len(truncated_df[0]))
#        print(type([zero_embeddings]*(question_size-len(truncated_df))))
#        print(np.array([zero_embeddings]*(question_size - len(truncated_df))).shape)
        if(len(truncated_df)!=30):
            truncated_df = np.concatenate((truncated_df, np.array([zero_embeddings]*(question_size- len(truncated_df)))))
#        truncated_df.append(zero_embeddings*(question_size - len(truncated_df)))
        return truncated_df



In [4]:
def batch_gen(n_batches,y,all_embeddings):
#    print("bg")
    while True: 
        for i in range(n_batches):
#            print("for loop")
            embeddings_list = all_embeddings[i*batch_size:(i+1)*batch_size] 
#           print("el",embeddings_list.shape,all_embeddings.shape,embeddings_list[0].shape)
            concat_embed_ques = np.array([get_concatenated_embeddings(ques) for ques in embeddings_list])
            #print(i,concat_embed_ques.shape)
            yield concat_embed_ques, np.array(y[i*batch_size:(i+1)*batch_size])


In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
all_embeddings = pickle.load(open("merged_embeddings_train","rb"))
#all_embeddings_test = pickle.load(open("merged_embeddings_test","rb"))	
#print(all_embeddings.shape, all_embeddings[0].shape)
#print(type(all_embeddings),type(all_embeddings[0]))
all_embeddings_train, all_embeddings_test,all_y_train,all_y_test = train_test_split(all_embeddings, train_df["target"][0:all_embeddings.shape[0]], test_size = 0.30, random_state = 42)
embeddings_train,embeddings_val,y_train,y_val= train_test_split(all_embeddings_train,all_y_train,test_size=0.20)
n_batches = math.ceil(len(y_train)/batch_size)
print("train set",len(y_train))
print("test set",len(all_y_test))
print("val set",len(y_val))
bg = batch_gen(n_batches,y_train,embeddings_train)
val_vects = np.array([get_concatenated_embeddings(val_emb) for val_emb in embeddings_val][:3000])
val_y = np.array(y_val[:3000])


train set 112000
test set 60000
val set 28000


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Bidirectional,CuDNNGRU,BatchNormalization,Activation,Dropout,Flatten
model = Sequential()
model.add(Bidirectional(CuDNNGRU(64, return_sequences=True),
                        input_shape=(30, 300)))
print(model.layers[0].output)
#model.add(Dense(128, activation="relu"))
#print(model.layers[1].output)
#model.add(Dense(64, activation="relu"))
#print(model.layers[2].output)
#model.add(BatchNormalization(momentum=0.1, epsilon=1e-05))
#print(model.layers[3].output)
#model.add(Activation("relu"))
#print(model.layers[4].output)
#model.add(Dropout(0.25))
#print(model.layers[5].output)
#model.add(Dense(16,bias=True))
#print(model.layers[6].output)
#model.add(BatchNormalization(momentum=0.1, epsilon=1e-05))
#print(model.layers[7].output)
#model.add(Activation("relu"))
#print(model.layers[8].output)
#model.add(Dropout(0.1))
#print(model.layers[9].output)
model.add(Flatten())
print(model.layers[1].output)
model.add(Dense(1,bias=True))
print(model.layers[2].output)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Tensor("bidirectional_1/concat_2:0", shape=(?, 30, 128), dtype=float32)
Tensor("flatten_1/Reshape:0", shape=(?, ?), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 1), dtype=float32)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, use_bias=True)`


In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
model.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f3ce51dbe80>>

In [ ]:
model.fit_generator(bg, epochs=10,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 26s 26ms/step - loss: 0.2927 - acc: 0.8848 - val_loss: 0.2645 - val_acc: 0.8773
Epoch 2/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4144 - acc: 0.6820 - val_loss: 0.5352 - val_acc: 0.3877
Epoch 3/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.8642 - acc: 0.1171 - val_loss: 0.9711 - val_acc: 0.0120
Epoch 4/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.9802 - acc: 0.0070 - val_loss: 0.9939 - val_acc: 3.3333e-04
Epoch 5/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.9961 - acc: 0.0587 - val_loss: 0.9312 - val_acc: 0.0377
Epoch 6/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.9248 - acc: 0.0401 - val_loss: 0.9939 - val_acc: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 23s 23ms/step - loss: 1.0058 - acc: 4.6875e-05 - val_loss: 0.9939 - val_

In [ ]:
#test_vects = np.array([get_concatenated_embeddings(test_emb) for test_emb in all_embeddings_test])
#test_y = np.array(y_test[:3000])
test_gen=batch_gen(n_batches,all_y_test,all_embeddings_test)
scores=model.evaluate_generator(test_gen,steps=400,verbose=1)
print("Accuracy", scores[1])


156/400 [==========>...................] - ETA: 2s